In [19]:
# import dependencies
import geojson

In [20]:
# path to geojson file
file_path = '../docs/resources/geojson/hike_kalalau.geojson'

# define the coordinates to cut the trail / road / track
# in the format (longitude, latitude)
endpoint = (-159.597593, 22.208304)

In [21]:
# handle floating point imprecision
def coordinates_match(coord1, coord2, tolerance=1e-4):
    return abs(coord1[0] - coord2[0]) <= tolerance and abs(coord1[1] - coord2[1]) <= tolerance


# function to filter the trail up to the endpoint
def filter_trail(feature, endpoint):
    filtered_coords = []
    for coord in feature["geometry"]["coordinates"]:
        filtered_coords.append(coord)
        if coordinates_match(coord, endpoint):  # stop at endpoint
            break
    return filtered_coords

In [22]:
# load GeoJSON
with open(file_path, "r") as file:
    data = geojson.load(file)

In [23]:
# process the GeoJSON
filtered_features = []
for feature in data["features"]:
    if feature["geometry"]["type"] == "LineString":
        filtered_coords = filter_trail(feature, endpoint)
        if filtered_coords:
            # create new feature with the filtered geometry
            new_feature = geojson.Feature(
                geometry=geojson.LineString(filtered_coords),
                properties=feature["properties"],
            )
            filtered_features.append(new_feature)

In [24]:
# save
filtered_geojson = geojson.FeatureCollection(filtered_features)
with open(file_path, "w") as file:
    geojson.dump(filtered_geojson, file, indent=2)

print("Filtered GeoJSON saved!")

Filtered GeoJSON saved!
